# Creating batch to test partial feedback setting

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('../')

In [3]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm
import source.players.dmd as dmd

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout
from random import shuffle

In [4]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [6]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)
reload(dmd)

<module 'source.players.dmd' from '..\\source\\players\\dmd.py'>

### Creating configurations to save into a csv file

In [7]:
time_horizon = 100
T = 6
targets = util.gen_tar_with_len(T)
values = tuple((v, v) for v in targets)
print(values)
feedback_prob = [1 for i in range(T)]
observabilities = [1 for i in range(T)]
feedback_type = "mab"
distributions = []
for i in range(19):
    distributions.append(tuple(util.gen_distr(T)))
print(distributions)
chosen_distr = tuple(util.gen_distr(T))
print("chosen = ", chosen_distr)

((0.809, 0.809), (0.842, 0.842), (0.933, 0.933), (0.875, 0.875), (0.986, 0.986), (0.876, 0.876))
[(0.15, 0.116, 0.139, 0.433, 0.014, 0.148), (0.231, 0.125, 0.164, 0.055, 0.066, 0.359), (0.2672672672672673, 0.2122122122122122, 0.07907907907907907, 0.12212212212212212, 0.06606606606606608, 0.25325325325325326), (0.006993006993006994, 0.1228771228771229, 0.1098901098901099, 0.6083916083916084, 0.025974025974025976, 0.1258741258741259), (0.046, 0.398, 0.024, 0.334, 0.056, 0.142), (0.237, 0.001, 0.174, 0.054, 0.249, 0.285), (0.4744744744744745, 0.0890890890890891, 0.11011011011011013, 0.1811811811811812, 0.13913913913913917, 0.006006006006006007), (0.177, 0.316, 0.198, 0.099, 0.104, 0.106), (0.1018981018981019, 0.15084915084915085, 0.17882117882117884, 0.25974025974025977, 0.18481518481518483, 0.12387612387612389), (0.07, 0.06, 0.152, 0.497, 0.111, 0.11), (0.02, 0.171, 0.292, 0.187, 0.151, 0.179), (0.33, 0.285, 0.27, 0.039, 0.034, 0.042), (0.020020020020020023, 0.19619619619619622, 0.491491

In [9]:
g = game.GameWithObservabilities(values, time_horizon, observabilities, feedback_prob, feedback_type = "mab")
g.attackers = [1]
g.defenders = [0]
d = [bm.FB(g, 0)]
prof = []
for distr in distributions:
    prof.append(attackers.StochasticAttacker(g, 1, 1, *distr))
prof.append(attackers.StochasticAttacker(g, 1, 1, chosen_distr))
prof.append(attackers.StackelbergAttacker(g, 1, 1))
a = [attackers.StochasticAttacker(g, 1, 1, chosen_distr)]
shuffle(prof)
print("Real attacker is: ", a[0])
g.set_players(d, a, prof)

Real attacker is:  sto1-(0.1898101898101898, 0.05894105894105893, 0.13486513486513485, 0.045954045954045945, 0.33966033966033965, 0.23076923076923075)


In [10]:
def print_header(targets, profiles):
    return "Name,T," + ",".join(str(i) for i in range(len(targets))) + ",Defender,Attacker," + ",".join(["Profile" for x in range(len(profiles))]) + "," + ",".join(["Obs" + str(x) for x in range(len(targets))]) + "," + ",".join(["Feed_prob" + str(x) for x in range(len(targets))]) + ",Feed_type" + "\n" 

In [11]:
def print_row(targets, time_horizon, d, a, p, obs, fprob, ftype):
    return ",".join([str(i) for i in ([d+"_vs_"+a] + [time_horizon] + targets + [d] + [a] + [x for x in p] + [o for o in obs] + [fp for fp in fprob] + [ftype]) ])+"\n"

In [12]:
(print_header(targets, prof))

'Name,T,0,1,2,3,4,5,Defender,Attacker,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Obs0,Obs1,Obs2,Obs3,Obs4,Obs5,Feed_prob0,Feed_prob1,Feed_prob2,Feed_prob3,Feed_prob4,Feed_prob5,Feed_type\n'

In [13]:
defender =["FB1"]

In [14]:
profile_strings = []
for p in prof:
    profile_strings.append(util.print_adv(p))

In [15]:
experiments_folder = "../NewExperiments"
exp_subfolder = "/FB MAB feedback"
batch_name = "fb_mab_feedback"
batch_file = batch_name + ".csv"
batch_path = experiments_folder + exp_subfolder + "/" + batch_file

### Generating the csv and checking everything is fine

In [16]:
with open(batch_path, "w+") as f:
    f.write(print_header(targets, prof))
    f.write(print_row(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings,observabilities,feedback_prob,feedback_type))

In [17]:
pd.read_csv(batch_path)

,,,,,,,,,,,,,,,Name,T,0,1,2,3,4,5,Defender,Attacker,...,Obs3,Obs4,Obs5,Feed_prob0,Feed_prob1,Feed_prob2,Feed_prob3,Feed_prob4,Feed_prob5,Feed_type
FB1_vs_sto1-(0.1898101898101898,0.058941,0.134865,0.045954,0.33966,0.23076923076923075),100,0.809,0.842,0.933,0.875,0.986,0.876,FB1,sto1-(0.1898101898101898,0.058941,0.134865,0.045954,0.33966,0.23076923076923075),sto1-0.1018981018981019-0.15084915084915085-0....,sto1-0.020020020020020023-0.19619619619619622-...,sto1-0.15-0.116-0.139-0.433-0.014-0.148,sto1-0.33-0.285-0.27-0.039-0.034-0.042,sto1-0.046-0.398-0.024-0.334-0.056-0.142,...,1,1,1,1,1,1,1,1,1,mab
